In [9]:
from datasets import load_dataset
from openai import OpenAI
from  dotenv import load_dotenv
import numpy as np

load_dotenv()
client = OpenAI()


In [2]:
ds_core = load_dataset("BASF-AI/ChemHotpotQARetrieval", "default")
ds_corpus = load_dataset("BASF-AI/ChemHotpotQARetrieval", "corpus")
ds_queries = load_dataset("BASF-AI/ChemHotpotQARetrieval", "queries")

In [18]:
ds_core['train']['scre']

ValueError: Column 'scre' doesn't exist.

In [5]:
def get_embedding(text):
    response = client.embeddings.create(
        input=text,
        model="text-embedding-3-small"
    )
    return response.data[0].embedding

In [ ]:
df_core

In [6]:
corpus_embeddings = [get_embedding(text) for text in ds_corpus['corpus']['text']]

In [10]:
def save_embeddings(embeddings, filename):
    np.save(filename, embeddings)

In [11]:
save_embeddings(corpus_embeddings, "corpus_embeddings.npy")

In [12]:
queries_embeddings  = [get_embedding(text) for text in ds_queries['queries']['text']]

In [13]:
save_embeddings(corpus_embeddings, "corpus_embeddings.npy")

In [ ]:
def ndcd_at_10(predictions, ground_truths):
    """
    Calculate the Normalized Discounted Cumulative Gain (NDCG) at rank 10.
    """
    dcg = 0.0
    idcg = 0.0

    for i in range(min(10, len(predictions))):
        if predictions[i] in ground_truths:
            dcg += 1 / (i + 1)
        idcg += 1 / (i + 1)

    return dcg / idcg if idcg > 0 else 0.0

In [ ]:

 ds_core['train'].to_pandas().to_csv("chemretrieval_bench_train.csv", index=False)

Dataset({
    features: ['query-id', 'corpus-id', 'score'],
    num_rows: 187
})

In [23]:
queries_embeddings = np.array(queries_embeddings)
corpus_embeddings = np.array(corpus_embeddings)

In [25]:
results = []
for i, row in ds_core['train'].to_pandas().iterrows():
    query = row['query-id']
    ground_truths = row['corpus-id']

    # retrieve the now using the embeddings
    predictions = []  
    # cosine similarity or any other retrieval method can be applied here
    print(query, ground_truths)
    for j, corpus_row in ds_corpus['corpus'].to_pandas().iterrows():
        similarity = np.dot(queries_embeddings[query], corpus_embeddings[corpus_row['id']])
        predictions.append((corpus_row['id'], similarity))
    
    predictions.sort(key=lambda x: x[1], reverse=True)
    predictions = [pred[0] for pred in predictions[:10]]  # Top
    ndcg = ndcd_at_10(predictions, ground_truths)
    results.append({
        'query-id': query,
        'ndcg_at_10': ndcg
    })


5ae75ace5542997b22f6a6f3 3395507


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices